# Text2Cypher Graph Agent Demo

In [17]:
import os
import sys
sys.path.append("../")

from dotenv import load_dotenv

load_dotenv()

from neo4j_graphrag.retrievers import Text2CypherRetriever
from langchain_openai.chat_models import ChatOpenAI

from neo4j import GraphDatabase
from neo4j_graphrag.schema import get_schema

from src.ps_genai_agents.prompts import (
    create_cypher_prompt,
)
from neo4j import Record
from neo4j_graphrag.types import RetrieverResultItem

In [18]:
def _format_schema(schema: str) -> str:
    """
    Formats Cypher for use in LangChain's Example Templates.
    This involves replacing '{' with '{{' and '}' with '}}'.
    """

    schema = schema.replace("{", "{{")
    return schema.replace("}", "}}")

In [19]:
chat_llm = ChatOpenAI(model="gpt-4o")
driver = GraphDatabase.driver(
    uri=os.environ.get("NEO4J_URI", ""),
    auth=(os.environ.get("NEO4J_USERNAME"), os.environ.get("NEO4J_PASSWORD")),
)

text2cypher_prompt = create_cypher_prompt(
    graph_schema=_format_schema(get_schema(driver=driver)),
    examples_yaml_path="../data/iqs/queries/queries.yml",
)

In [20]:
print(text2cypher_prompt)

You are an expert Neo4j Cypher translator who understands the question in english and convert to Cypher strictly based on the Neo4j Schema provided and following the instructions below:
    <instructions>
    * Use aliases to refer the node or relationship in the generated Cypher query
    * Generate Cypher query compatible ONLY for Neo4j Version 5
    * Do not use EXISTS in the cypher. Use alias when using the WITH keyword
    * Only use SIZE when checking the size of a list
    * Use only Nodes and relationships mentioned in the schema
    * Always do a case-insensitive and fuzzy search for any properties related search. Eg: to search for a Company name use `toLower(c.name) contains 'neo4j'`
    * Cypher is NOT SQL. So, do not mix and match the syntaxes
    * Ensure that null results are filtered out before running aggregations!
    * Do not include any header
    * Do not wrap in backticks (```)
    * Return only Cypher
    </instructions>

    Strictly use this Schema for Cypher ge

In [24]:
def record_formatter(record: Record) -> RetrieverResultItem:
        """
        Best effort to guess the node-to-text method. Inherited classes
        can override this method to implement custom text formatting.
        """
        return RetrieverResultItem(content=record, metadata=record.get("metadata"))

In [25]:
retriever = Text2CypherRetriever(driver=driver, llm=chat_llm, custom_prompt=text2cypher_prompt, result_formatter=record_formatter)

In [41]:
res = retriever.search(query_text="how many vehicles are there?")

In [42]:
res.items[0].content.data()

{'totalVehicles': 12}

In [50]:
res = retriever.search(query_text="What are the male to female ratio for complaints on Honda Civics")

In [51]:
[x.content.data() for x in res.items]

[{'males': 612, 'females': 276, 'maleToFemaleRatio': 2.217391304347826}]

In [43]:
res = retriever.search(query_text="What are the top 10 complaints on Honda Civic for people over age 40")

In [49]:
[x.content.data() for x in res.items]

[{'problem': 'DRA16: Lane departure warning/lane keeping assistance - Alerts annoying/bothersome',
  'total': 31},
 {'problem': 'EXT30: Excessive road noise', 'total': 13},
 {'problem': 'EXT22: Insufficient ground clearance', 'total': 12},
 {'problem': 'INFO20: Not enough power plugs/USB ports', 'total': 12},
 {'problem': 'INT27: Interior materials scuff/soil easily', 'total': 12},
 {'problem': 'INFO08: Touchscreen/display screen - DTU', 'total': 10},
 {'problem': 'DRA03: Adaptive cruise control - Alerts annoying/bothersome',
  'total': 10},
 {'problem': 'DRA05: Parking cameras - Lens gets dirty too easily',
  'total': 9},
 {'problem': "CLMT08: Windows fog up excessively/don't clear quickly",
  'total': 8},
 {'problem': "FCD35: OEM smartphone app - Doesn't connect/connects inconsistently/slow to connect",
  'total': 7}]

In [57]:
res = retriever.search(query_text="^")

In [59]:
res.metadata.get("cypher")

"MATCH (q:Question {id: 10})<-[:HAS_QUESTION]-(v:Verbatim)\n  WHERE v.model = 'RDX'\nWITH v\nLIMIT 50\nMATCH (v)-[:HAS_CATEGORY]->(c:Category)\nRETURN v.verbatim AS verbatim, COLLECT(DISTINCT c.id) AS categories\nLIMIT 50"